There are two ways of collecting and analyzing NGINX logs in AWS:

1. Download all the logentries from Cloudwatch filtered on `StreamPrefix='application/nginx'`` and analyze them locally
2. Execute a query with boto3 and analyze the results

In [12]:
%pip install boto3 prettytable pandas

4778.15s - pydevd: Sending message related to process being replaced timed-out after 5 seconds

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import boto3
import subprocess
import json
import os
import time
import pandas as pd
from datetime import datetime, timezone

In [14]:
def set_aws_credentials(profile, region_name='us-east-1'):
    result = subprocess.run(f"aws-vault exec {profile} --json", shell=True, capture_output=True)
    credentials = json.loads(result.stdout)

    # Create a session with the retrieved credentials
    session = boto3.session.Session(
        aws_access_key_id=credentials['AccessKeyId'],
        aws_secret_access_key=credentials['SecretAccessKey'],
        aws_session_token=credentials['SessionToken'],
        region_name=region_name        
    )

    return session

# Use the function with your profile to get a session
aws_session = set_aws_credentials('acl-production', 'eu-central-1')

# Create clients using the session
sts_client = aws_session.client('sts')
logs_client = aws_session.client('logs')

# Example usage of the clients
account_id = sts_client.get_caller_identity()["Account"]
print("Current AWS Account ID:", account_id)

Current AWS Account ID: 707785685172


In [22]:
# select 'projects-main' log group and collect log streams starting with 'application/nginx/'
# get log entries based on a time frame ginven in parameters

def get_log_entries(logs_client, log_group_name, log_stream_prefix, start_time, end_time):
    paginator = logs_client.get_paginator('filter_log_events')
    response_iterator = paginator.paginate(
        logGroupName=log_group_name,
        logStreamNamePrefix=log_stream_prefix,
        startTime=start_time,
        endTime=end_time,
        PaginationConfig={'MaxItems': 1000}
    )
    log_events = []
    for response in response_iterator:
        log_events += response['events']
    return log_events

log_group_name = 'projects-main'
log_stream_name_prefix = 'application/nginx'

#start_time = int(time.time()) - 3600 * 24 * 1 # 1 days ago
#end_time = int(time.time())

start_time = int(datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc).timestamp() * 1000)  # 1st Jan 2024
end_time = int(datetime(2024, 1, 3, 0, 0, 0, tzinfo=timezone.utc).timestamp() * 1000)   # 2nd Jan 2024

log_events = get_log_entries(logs_client, log_group_name, log_stream_name_prefix, start_time, end_time)
print("Number of log events:", len(log_events))

# print the top 100 entries
for log_event in log_events[:100]:
    print(log_event['message'])


Number of log events: 1000
{"time_local":"01/Jan/2024:00:00:00 +0000","client":"10.185.254.61","ACL_Cloudfront":"","method":"GET","scheme":"http","host":"projects-eu.highbond.com","request":"GET /api/v3/mitigations/2309919?org_id=38645 HTTP/1.1","request_length":"3822","status": "200","bytes_sent":"1307","body_bytes_sent":"278","referrer":"","user_agent":"python-requests/2.26.0","request_time":"0.097","remote_user":"podriscoll@diligent.com","x_nginx_id":"76a94bab6b8fa4182d8e097741ba4ff8","x_request_id":"d71b31565aae57dc15e60222db9eff26","upstream_status":"200","upstream_response_time":"0.096","upstream_connect_time":"0.000","upstream_header_time":"0.096"}
{"time_local":"01/Jan/2024:00:00:00 +0000","client":"10.185.255.160","ACL_Cloudfront":"","method":"GET","scheme":"http","host":"projects-eu.highbond.com","request":"GET /api/v3/mitigations/2317558?org_id=38645 HTTP/1.1","request_length":"3822","status": "200","bytes_sent":"1307","body_bytes_sent":"278","referrer":"","user_agent":"pyth